# Simple Prompt Optimizer

In [10]:
#!pip install dspy-ai
#!pip install dspy-ai

In [11]:
import sys
import os
import dspy 
from common.my_settings import MySettings  
from common.utils import md
from common.llm_client_factory import LlmClientFactory
from dspy_utils.dspy_helpers import md_dspy

settings = MySettings().get()

In [12]:

lm = dspy.LM(
    'gpt-4.1',  # Changed from gpt-4o to gpt-3.5-turbo
    model_type='chat', 
    cache=False, 
    api_key=os.getenv("OPENAI_API_KEY")
)

dspy.configure(lm=lm)

In [13]:
import os
import re
import dspy

# Check version
import importlib.metadata
print("DSPy version:", importlib.metadata.version("dspy-ai"))

# ---------------------------
# 1) Configure LLM
# ---------------------------
# Use Databricks secret or env var for your key
# api_key = dbutils.secrets.get("scope-name", "openai-key")  # or os.getenv("OPENAI_API_KEY")
# dspy.configure(lm=dspy.OpenAI(model="gpt-4o-mini", api_key=api_key))
dspy.configure(lm=lm)

# ---------------------------
# 2) Define Signature and Module
# ---------------------------
class Summarize(dspy.Signature):
    text = dspy.InputField()
    summary = dspy.OutputField()

class Summarizer(dspy.Module):
    def __init__(self):
        super().__init__()
        self.predict = dspy.Predict(Summarize)
    def forward(self, text):
        return self.predict(text=text)

# ---------------------------
# 3) Prepare examples (must use dspy.Example + .with_inputs)
# ---------------------------
# trainset = [
#     dspy.Example(
#         text="AI is transforming industries by automating tasks.",
#         summary="AI automates tasks across industries."
#     ).with_inputs("text"),
#     dspy.Example(
#         text="Climate change causes rising seas and extreme weather.",
#         summary="Climate change leads to rising seas and extreme weather."
#     ).with_inputs("text"),
# ]

# trainset = [
#     dspy.Example(text="AI is transforming industries by automating tasks.", summary="AI automates tasks across industries.").with_inputs("text"),
#     dspy.Example(text="Climate change causes rising seas and extreme weather.", summary="Climate change leads to rising seas and extreme weather.").with_inputs("text"),
    
#     # Short summaries (2 words or fewer)
#     dspy.Example(text="Electric vehicles are becoming more popular due to environmental concerns.", summary="EV adoption").with_inputs("text"),
#     dspy.Example(text="Remote work has changed how companies manage productivity.", summary="Remote shift").with_inputs("text"),
#     dspy.Example(text="Cryptocurrency markets are highly volatile and speculative.", summary="Crypto volatility").with_inputs("text"),
#     dspy.Example(text="Online education platforms are expanding access to learning.", summary="E-learning").with_inputs("text"),
#     dspy.Example(text="Social media influences public opinion and behavior.", summary="Influence").with_inputs("text"),
#     dspy.Example(text="Renewable energy sources are replacing fossil fuels.", summary="Green energy").with_inputs("text"),
#     dspy.Example(text="Cybersecurity threats are increasing in frequency and complexity.", summary="Cyber threats").with_inputs("text"),
#     dspy.Example(text="Space exploration is advancing with private sector involvement.", summary="Space race").with_inputs("text"),

#     # Longer summaries
#     dspy.Example(text="Machine learning models are improving medical diagnostics.", summary="ML enhances medical diagnostics.").with_inputs("text"),
#     dspy.Example(text="Supply chain disruptions have impacted global trade.", summary="Global trade affected by supply chain issues.").with_inputs("text"),
#     dspy.Example(text="Smartphones have become essential tools for communication and productivity.", summary="Smartphones are vital for modern communication and work.").with_inputs("text"),
#     dspy.Example(text="Augmented reality is being used in retail to enhance customer experience.", summary="AR improves retail customer experience.").with_inputs("text"),
#     dspy.Example(text="5G networks promise faster connectivity and new applications.", summary="5G enables faster and broader connectivity.").with_inputs("text"),
#     dspy.Example(text="Digital banking is replacing traditional financial services.", summary="Digital banking transforms financial services.").with_inputs("text"),
#     dspy.Example(text="Climate policies are shaping national energy strategies.", summary="Climate policy influences energy planning.").with_inputs("text"),
#     dspy.Example(text="Wearable technology is helping people monitor their health.", summary="Wearables support health monitoring.").with_inputs("text"),
#     dspy.Example(text="Data privacy regulations are evolving across regions.", summary="Data privacy laws are becoming more stringent.").with_inputs("text"),
#     dspy.Example(text="Artificial intelligence is being used to personalize marketing.", summary="AI personalizes marketing strategies.").with_inputs("text"),
# ]

# 75 in total
trainset = [
    # 38 short summaries (2 words or fewer)
    dspy.Example(text="AI is transforming industries by automating tasks.", summary="AI automation").with_inputs("text"),
    dspy.Example(text="Climate change causes rising seas and extreme weather.", summary="Climate impact").with_inputs("text"),
    dspy.Example(text="Electric vehicles are becoming more popular due to environmental concerns.", summary="EV growth").with_inputs("text"),
    dspy.Example(text="Remote work has changed how companies manage productivity.", summary="Remote shift").with_inputs("text"),
    dspy.Example(text="Cryptocurrency markets are highly volatile and speculative.", summary="Crypto risk").with_inputs("text"),
    dspy.Example(text="Online education platforms are expanding access to learning.", summary="E-learning").with_inputs("text"),
    dspy.Example(text="Social media influences public opinion and behavior.", summary="Social influence").with_inputs("text"),
    dspy.Example(text="Renewable energy sources are replacing fossil fuels.", summary="Green energy").with_inputs("text"),
    dspy.Example(text="Cybersecurity threats are increasing in frequency and complexity.", summary="Cyber threats").with_inputs("text"),
    dspy.Example(text="Space exploration is advancing with private sector involvement.", summary="Space race").with_inputs("text"),
    dspy.Example(text="Machine learning models are improving medical diagnostics.", summary="ML diagnostics").with_inputs("text"),
    dspy.Example(text="Supply chain disruptions have impacted global trade.", summary="Trade disruption").with_inputs("text"),
    dspy.Example(text="Smartphones have become essential tools for communication and productivity.", summary="Smartphones").with_inputs("text"),
    dspy.Example(text="Augmented reality is being used in retail to enhance customer experience.", summary="AR retail").with_inputs("text"),
    dspy.Example(text="5G networks promise faster connectivity and new applications.", summary="5G rollout").with_inputs("text"),
    dspy.Example(text="Digital banking is replacing traditional financial services.", summary="Digital banking").with_inputs("text"),
    dspy.Example(text="Climate policies are shaping national energy strategies.", summary="Climate policy").with_inputs("text"),
    dspy.Example(text="Wearable technology is helping people monitor their health.", summary="Wearables").with_inputs("text"),
    dspy.Example(text="Data privacy regulations are evolving across regions.", summary="Privacy laws").with_inputs("text"),
    dspy.Example(text="Artificial intelligence is being used to personalize marketing.", summary="AI marketing").with_inputs("text"),
    dspy.Example(text="Biometric authentication is becoming more common in mobile devices.", summary="Biometrics").with_inputs("text"),
    dspy.Example(text="Cloud computing enables scalable infrastructure for businesses.", summary="Cloud tech").with_inputs("text"),
    dspy.Example(text="Drones are being used for delivery and surveillance.", summary="Drone use").with_inputs("text"),
    dspy.Example(text="Genetic editing tools like CRISPR are revolutionizing medicine.", summary="Gene editing").with_inputs("text"),
    dspy.Example(text="Digital twins are used to simulate real-world systems.", summary="Digital twins").with_inputs("text"),
    dspy.Example(text="Quantum computing promises exponential processing power.", summary="Quantum leap").with_inputs("text"),
    dspy.Example(text="Voice assistants are becoming more integrated into daily life.", summary="Voice tech").with_inputs("text"),
    dspy.Example(text="Facial recognition is used in security and marketing.", summary="Face ID").with_inputs("text"),
    dspy.Example(text="3D printing is transforming manufacturing and prototyping.", summary="3D printing").with_inputs("text"),
    dspy.Example(text="Blockchain is being explored for secure digital transactions.", summary="Blockchain").with_inputs("text"),
    dspy.Example(text="Smart cities use data to improve urban living.", summary="Smart cities").with_inputs("text"),
    dspy.Example(text="Digital art is gaining popularity through NFTs.", summary="NFT art").with_inputs("text"),
    dspy.Example(text="Subscription models are changing how we consume media.", summary="Subscriptions").with_inputs("text"),
    dspy.Example(text="Food delivery apps are reshaping the restaurant industry.", summary="Food apps").with_inputs("text"),
    dspy.Example(text="Electric scooters are popular in urban transportation.", summary="E-scooters").with_inputs("text"),
    dspy.Example(text="Crowdfunding platforms help launch creative projects.", summary="Crowdfunding").with_inputs("text"),
    dspy.Example(text="Digital detoxes are becoming more common.", summary="Digital detox").with_inputs("text"),
    dspy.Example(text="Gamification is used to boost engagement in apps.", summary="Gamification").with_inputs("text"),

    # 12 longer summaries
    dspy.Example(text="AI is helping doctors detect diseases earlier and more accurately.", summary="AI improves early disease detection in healthcare.").with_inputs("text"),
    dspy.Example(text="Climate change is accelerating the melting of polar ice caps.", summary="Polar ice caps are melting faster due to climate change.").with_inputs("text"),
    dspy.Example(text="Electric vehicles reduce emissions and dependence on fossil fuels.", summary="EVs help reduce carbon emissions and oil reliance.").with_inputs("text"),
    dspy.Example(text="Remote work has led to changes in office space demand.", summary="Remote work is reshaping commercial real estate needs.").with_inputs("text"),
    dspy.Example(text="Cryptocurrency adoption is growing despite regulatory uncertainty.", summary="Crypto adoption continues amid unclear regulations.").with_inputs("text"),
    dspy.Example(text="Online education is bridging gaps in access to quality learning.", summary="E-learning expands access to quality education worldwide.").with_inputs("text"),
    dspy.Example(text="Social media platforms are under scrutiny for misinformation.", summary="Misinformation on social media is drawing regulatory attention.").with_inputs("text"),
    dspy.Example(text="Renewable energy investments are increasing globally.", summary="Global investment in renewables is on the rise.").with_inputs("text"),
    dspy.Example(text="Cybersecurity breaches are prompting stronger data protection laws.", summary="Data breaches are driving stricter cybersecurity regulations.").with_inputs("text"),
    dspy.Example(text="Space tourism is becoming a reality for private citizens.", summary="Private space travel is emerging as a new industry.").with_inputs("text"),
    dspy.Example(text="Machine learning is being used to predict customer behavior.", summary="ML helps businesses forecast customer actions and preferences.").with_inputs("text"),
    dspy.Example(text="Supply chain resilience is a top priority post-pandemic.", summary="Companies are focusing on building resilient supply chains.").with_inputs("text"),
]


# ---------------------------
# 4) Simple metric
# ---------------------------
def simple_metric(example, pred, trace=None):
    print("Simple Metric: example: ", example, ", pred:", pred)
    print("pred.summary: ", pred.summary)
    # def toks(s): return set(re.findall(r"\w+", (s or "").lower()))
    # gt, pr = toks(example.summary), toks(pred.summary)
    # result = len(gt & pr) / len(gt) if gt else 0.0
    # print("Simple metric:result:", result)
    # return result  
    sentence = pred.summary
    word_count = len(sentence.split())
    print("Simple metric: word count:", word_count, " less than two: ", word_count <= 2)
    return word_count <= 2


# ---------------------------
# 5) Baseline
# ---------------------------
base = Summarizer()
print("Baseline:", base("Prompt optimization improves LLM outputs.").summary)

# ---------------------------
# 6) Optimize with BootstrapFewShot
# ---------------------------
from dspy.teleprompt import BootstrapFewShot
teleprompter = BootstrapFewShot(metric=simple_metric, max_bootstrapped_demos=4, max_labeled_demos=4)

optimized_program = teleprompter.compile(
    student=Summarizer(),
    trainset=trainset
)

# ---------------------------
# 7) Compare after optimization
# ---------------------------
print("Optimized:", optimized_program("Prompt optimization improves LLM outputs.").summary)


DSPy version: 3.0.1


Baseline: Optimizing prompts leads to better results from large language models (LLMs).


  2%|▏         | 1/50 [00:00<00:39,  1.24it/s]

Simple Metric: example:  Example({'text': 'AI is transforming industries by automating tasks.', 'summary': 'AI automation'}) (input_keys={'text'}) , pred: Prediction(
    summary='AI automation'
)
pred.summary:  AI automation
Simple metric: word count: 2  less than two:  True


  4%|▍         | 2/50 [00:01<00:39,  1.20it/s]

...

 98%|█████████▊| 49/50 [00:46<00:00,  1.01it/s]

Simple Metric: example:  Example({'text': 'Machine learning is being used to predict customer behavior.', 'summary': 'ML helps businesses forecast customer actions and preferences.'}) (input_keys={'text'}) , pred: Prediction(
    summary='ML for customer prediction'
)
pred.summary:  ML for customer prediction
Simple metric: word count: 4  less than two:  False


100%|██████████| 50/50 [00:47<00:00,  1.05it/s]

Simple Metric: example:  Example({'text': 'Supply chain resilience is a top priority post-pandemic.', 'summary': 'Companies are focusing on building resilient supply chains.'}) (input_keys={'text'}) , pred: Prediction(
    summary='Supply chain resilience is emphasized after the pandemic.'
)
pred.summary:  Supply chain resilience is emphasized after the pandemic.
Simple metric: word count: 8  less than two:  False
Bootstrapped 2 full traces after 49 examples for up to 1 rounds, amounting to 50 attempts.


Optimized: Optimizing prompts enhances large language model performance.


# 🔍 Why You Don’t See Prompt Changes

In DSPy, especially with BootstrapFewShot, the optimization process doesn't directly rewrite the prompt string (like "Use as few words as possible"). Instead, it:

Selects examples from your trainset that best guide the model toward satisfying your metric.
Constructs a few-shot prompt using those examples.
Does not modify the Signature instructions unless you explicitly use a module that supports prompt rewriting (like MIPROv2 with auto="rewrite").
So the prompt string in your Summarize class stays the same, but the few-shot context passed to the model changes.

In [17]:
# Prompts below are the recommended or most optimised "Few Shot Examples"
print(len(optimized_program.predict.demos), "demos in optimized program")
for i in optimized_program.predict.demos:
    print(i)
    #print(i.text, "->", i.summary)

4 demos in optimized program
Example({'augmented': True, 'text': 'AI is transforming industries by automating tasks.', 'summary': 'AI automation'}) (input_keys=None)
Example({'augmented': True, 'text': 'Cryptocurrency markets are highly volatile and speculative.', 'summary': 'Crypto volatility'}) (input_keys=None)
Example({'text': 'Remote work has led to changes in office space demand.', 'summary': 'Remote work is reshaping commercial real estate needs.'}) (input_keys={'text'})
Example({'text': 'Electric vehicles reduce emissions and dependence on fossil fuels.', 'summary': 'EVs help reduce carbon emissions and oil reliance.'}) (input_keys={'text'})
